# 1. Environment Setup
Installing dependencies and importing required libraries.

In [ ]:
!pip install nnunetv2

import os
import json
import shutil
from pathlib import Path
import numpy as np
from typing import List, Dict

print("✓ Dependencies installed and imported")


# 2. Experiment Configuration
Setting up paths, environment variables, and defining rare subjects.

In [ ]:
# Training configuration - Fold Selection
FOLD = 0  # Select fold (0, 1, 2, 3, 4)
CHECKPOINT_PATH = f"Enter your checkpoint path here" # Define your Checkpoint paths

# ------------------------------------------
# User Programmable Parameters
# ------------------------------------------
TRAINING_TIME_MINUTES = (11 * 60) + 45  # 11 hours 45 minutes
OVERSAMPLE_FACTOR = 3.0

# Define your dataset paths - MODIFY THESE TO MATCH YOUR KAGGLE PATHS
PREPROCESSED_PATH = "/kaggle/input/preprocessed-bonnfcd-flair/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR"
RAW_PATH = "/kaggle/input/preprocessed-bonnfcd-flair/nnUNet_raw_data_base/nnUNet_raw/Dataset002_BonnFCD_FLAIR"

# nnUNet environment variables
os.environ['nnUNet_raw'] = "/kaggle/input/preprocessed-bonnfcd-flair/nnUNet_raw_data_base/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/kaggle/input/preprocessed-bonnfcd-flair/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/kaggle/working/nnUNet_results"
if 'nnUNet_compile' not in os.environ:
    os.environ['nnUNet_compile'] = 'false' # Set default

# ------------------------------------------
# Configuration for Custom Modules
# ------------------------------------------
# Define the path to the custom modules directory
CUSTOM_MODULES_PATH = "custom_modules"
RARE_SUBJECTS_PATH = os.path.join(CUSTOM_MODULES_PATH, "rare_subjects.json")
SPLITS_PATH = os.path.join(CUSTOM_MODULES_PATH, "splits_final.json")

# Load rare subjects for validation checks
with open(RARE_SUBJECTS_PATH, "r") as f:
    RARE_SUBJECTS = json.load(f)

# CREATE TRAINER CONFIG JSON
trainer_config = {
    "oversample_factor": OVERSAMPLE_FACTOR,
    "max_time_minutes": TRAINING_TIME_MINUTES
}
config_path = os.path.join(CUSTOM_MODULES_PATH, "trainer_config.json")
with open(config_path, "w") as f:
    json.dump(trainer_config, f, indent=4)

print(f"✓ Configuration set")
print(f"  - Custom modules path: {CUSTOM_MODULES_PATH}")
print(f"  - Trainer config created at: {config_path}")
print(f"  - Oversample factor: {OVERSAMPLE_FACTOR}x")


# 3. Custom nnU-Net Components
Defining the custom DataLoader and Trainer classes to handle oversampling and safe training limits.

In [ ]:
def create_custom_dataloader_file():
    """
    Copy the custom DataLoader from custom_modules to the working directory
    """
    source = os.path.join(CUSTOM_MODULES_PATH, "custom_dataloader.py")
    target_dir = "/kaggle/working/custom_nnunet"
    os.makedirs(target_dir, exist_ok=True)
    target = os.path.join(target_dir, "custom_dataloader.py")
    
    shutil.copy2(source, target)
    print(f"✓ Custom DataLoader copied from {source} to {target}")

create_custom_dataloader_file()


In [ ]:
import nnunetv2
from pathlib import Path

def create_custom_trainer():
    """
    Copy the custom Trainer and config from custom_modules to:
    1. The working directory (for reference/import)
    2. The nnU-Net package directory (so it can be used by -tr argument)
    """
    target_dir = "/kaggle/working/custom_nnunet"
    os.makedirs(target_dir, exist_ok=True)
    
    # Files to copy
    files_to_copy = [
        "nnUNetTrainerOversampling.py",
        "rare_subjects.json",
        "trainer_config.json"
    ]
    
    # 1. Copy to working dir
    for file in files_to_copy:
        src = os.path.join(CUSTOM_MODULES_PATH, file)
        dst = os.path.join(target_dir, file)
        if os.path.exists(src):
            shutil.copy2(src, dst)
        else:
            print(f"⚠ Warning: {src} not found!")

    # 2. Copy to nnU-Net package location
    nnunet_trainers_dir = Path(nnunetv2.__file__).parent / "training" / "nnUNetTrainer"
    
    for file in files_to_copy:
        src = os.path.join(CUSTOM_MODULES_PATH, file)
        dst = nnunet_trainers_dir / file
        if os.path.exists(src):
            shutil.copy2(src, dst)
            
    print(f"✓ Custom Trainer deployed to:")
    print(f"  - {target_dir}")
    print(f"  - {nnunet_trainers_dir}")

create_custom_trainer()


# 4. Data Preparation
Copying the preprocessed dataset to the working directory.

In [ ]:
source = "/kaggle/input/preprocessed-bonnfcd-flair/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR"
dest = "/kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR"

print("Copying preprocessed data to writable location...")
print(f"From: {source}")
print(f"To: {dest}")

Path(dest).parent.mkdir(parents=True, exist_ok=True)
shutil.copytree(source, dest, dirs_exist_ok=True)

print(f"✓ Copied successfully")
os.environ['nnUNet_preprocessed'] = "/kaggle/working/nnUNet_preprocessed"
print(f"✓ Updated nnUNet_preprocessed path")


# 5. Advanced Customization (Augmentation & Splits)
Applying doubled data augmentation parameters and enforcing fixed cross-validation splits.

In [ ]:
import json
import math
import os

plans_path = "/kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR/nnUNetPlans.json"

# Load plans
with open(plans_path, "r") as f:
    plans = json.load(f)

# ----------------------------------------------------
# Ensure the augmentation keys exist
# ----------------------------------------------------
if "data_augmentation" not in plans:
    plans["data_augmentation"] = {}

if "spatial" not in plans["data_augmentation"]:
    plans["data_augmentation"]["spatial"] = {}

if "intensity" not in plans["data_augmentation"]:
    plans["data_augmentation"]["intensity"] = {}

# ----------------------------------------------------
# 🔥 Doubled spatial augmentations
# ----------------------------------------------------
# Rotation: ±60 degrees (Default was 30)
plans["data_augmentation"]["spatial"]["rotation"] = {
    "x": 60 * (math.pi / 180),
    "y": 60 * (math.pi / 180),
    "z": 60 * (math.pi / 180)
}

# Scale: Range [0.70, 1.50] (Default was [0.85, 1.25])
# Logic: Doubled the deviation from 1.0 (15%->30% down, 25%->50% up)
plans["data_augmentation"]["spatial"]["scale_range"] = [0.70, 1.50]

# Elastic Deformation: Still Disabled (Doubling None is None)
# If you want to force it on, you must add a dictionary here.
plans["data_augmentation"]["spatial"]["elastic_deform"] = None

# ----------------------------------------------------
# 🔥 Doubled intensity augmentations
# ----------------------------------------------------
# Brightness: Range [0.5, 1.5] (Default was [0.75, 1.25])
plans["data_augmentation"]["intensity"]["brightness"] = [0.5, 1.5]

# Contrast: Range [0.5, 1.5] (Default was [0.75, 1.25])
plans["data_augmentation"]["intensity"]["contrast"] = [0.5, 1.5]

# Gaussian Noise: Variance 0.2 (Default was 0.1)
plans["data_augmentation"]["intensity"]["gaussian_noise_std"] = 0.2

# Gamma: Range [0.4, 2.0] (Default was [0.7, 1.5])
# Logic: Doubled the deviation from 1.0
plans["data_augmentation"]["intensity"]["gamma_range"] = [0.4, 2.0]

# Save back
with open(plans_path, "w") as f:
    json.dump(plans, f, indent=4)

print("🔥 Augmentation parameters DOUBLED successfully!")
print("- Rotation: ±60°")
print("- Scale: 0.70 - 1.50")
print("- Noise/Contrast/Gamma: Deviation doubled")


In [ ]:
import json
import os

# ----------------------------------------------------
# 🧪 Custom 5-Fold Cross-Validation Splits
# ----------------------------------------------------
# Load splits from JSON
with open(SPLITS_PATH, "r") as f:
    custom_splits = json.load(f)

# Define the target path where nnU-Net expects the splits file
# It must be in the preprocessed directory of the specific task
preprocessed_dir = "/kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR"
splits_file = os.path.join(preprocessed_dir, "splits_final.json")

# Write the splits to the file
os.makedirs(preprocessed_dir, exist_ok=True)
with open(splits_file, "w") as f:
    json.dump(custom_splits, f, indent=4)

print(f"✅ Custom splits enforced successfully!")
print(f"  - Loaded from: {SPLITS_PATH}")
print(f"  - Written to: {splits_file}")
print(f"  - Total folds: {len(custom_splits)}")


# 6. Checkpoint Restoration
Copying the previous checkpoint logic to resume training.

In [ ]:
print("\n" + "="*70)
print("COPYING PREVIOUS CHECKPOINT")
print("="*70)

# Create results directory structure
results_dir = Path(f"/kaggle/working/nnUNet_results/Dataset002_BonnFCD_FLAIR/nnUNetTrainerOversampling__nnUNetPlans__3d_fullres/fold_{FOLD}")
results_dir.mkdir(parents=True, exist_ok=True)

# Define source
checkpoint_source = Path(CHECKPOINT_PATH)

if checkpoint_source.exists():
    print(f"\n✓ Found checkpoint source: {checkpoint_source}")

    # CASE 1: user provided a specific FILE (e.g., checkpoint_final.pth)
    if checkpoint_source.is_file():
        # Rename it to what nnU-Net expects for resuming: 'checkpoint_latest.pth'
        dest_name = "checkpoint_latest.pth"
        dest_item = results_dir / dest_name
        shutil.copy2(checkpoint_source, dest_item)
        print(f"  ✓ Copied file: {checkpoint_source.name} -> {dest_name}")

    # CASE 2: user provided a DIRECTORY
    elif checkpoint_source.is_dir():
        # Copy all files from that directory
        for item in checkpoint_source.iterdir():
            dest_item = results_dir / item.name
            if item.is_file():
                shutil.copy2(item, dest_item)
                print(f"  ✓ Copied: {item.name}")
            elif item.is_dir():
                shutil.copytree(item, dest_item, dirs_exist_ok=True)
                print(f"  ✓ Copied directory: {item.name}")
    
    # Check what we have now
    if (results_dir / "checkpoint_latest.pth").exists() or (results_dir / "checkpoint_final.pth").exists():
        print(f"\n✓ Checkpoint restoration successful.")
    else:
        print("\n⚠ Warning: No valid checkpoint file (latest/final) found in destination!")

else:
    print(f"\n✗ Checkpoint path not found: {CHECKPOINT_PATH}")
    print("Training will start from scratch")

print("="*70)

# 7. Training Execution
Launching the nnU-Net training command with the custom trainer and fold configuration.

In [ ]:
print("\n" + "="*70)
print("CONTINUING TRAINING WITH CUSTOM OVERSAMPLING")
print("="*70)
print(f"\nConfiguration:")
print(f"  - Dataset: 002 (BonnFCD_FLAIR)")
print(f"  - Configuration: 3d_fullres")
print(f"  - Fold: {FOLD}")
print(f"  - Trainer: nnUNetTrainerOversampling")
print(f"  - Rare subjects: {len(RARE_SUBJECTS)}")
print(f"  - Oversample factor: {OVERSAMPLE_FACTOR}x")
print(f"  - Continuing from previous checkpoint")
print("="*70 + "\n")

# Continue training - nnUNet will automatically detect and load the latest checkpoint
!nnUNetv2_train 002 3d_fullres {FOLD} -tr nnUNetTrainerOversampling -p nnUNetPlans --npz --c